In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# ====== 1. Generate Dataset ======
np.random.seed(42)
n_samples = 5000

data = {
    'age': np.random.randint(18, 80, n_samples),
    'credit_score': np.random.normal(650, 100, n_samples).astype(int),
    'balance': np.round(np.random.exponential(50000, n_samples), 2),
    'products_number': np.random.randint(1, 5, n_samples),
    'estimated_salary': np.round(np.random.uniform(20000, 150000, n_samples), 2),
    'country': np.random.choice(['Germany', 'France', 'Spain'], n_samples),
    'gender': np.random.choice(['Male', 'Female'], n_samples),
    'tenure': np.random.randint(0, 10, n_samples),
    'has_credit_card': np.random.choice([0, 1], n_samples),
    'is_active_member': np.random.choice([0, 1], n_samples)
}

df = pd.DataFrame(data)

# Create target variable
df['churn'] = (
    (df['balance'] > 80000) | 
    (df['credit_score'] < 550) |
    (df['products_number'] == 1) |
    ((df['age'] > 65) & (df['is_active_member'] == 0))
).astype(int)

print(f"Dataset: {len(df)} rows × {len(df.columns)} columns")
print(f"Churn rate: {df['churn'].mean():.2%}\n")

# ====== 2. Data Preprocessing ======
# Encode categorical variables
le_country = LabelEncoder()
le_gender = LabelEncoder()
df['country'] = le_country.fit_transform(df['country'])
df['gender'] = le_gender.fit_transform(df['gender'])

# Separate features and target
X = df.drop('churn', axis=1).values
y = df['churn'].values.reshape(-1, 1)

# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}\n")

# ====== 3. Build 3-Layer MLP with NumPy ======
class MLP:
    def __init__(self, input_size, hidden_size, output_size):
        # Xavier initialization for better convergence
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0/input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0/hidden_size)
        self.b2 = np.zeros((1, output_size))
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        return z * (1 - z)
    
    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)
        return self.a2
    
    def backward(self, X, y, output, learning_rate):
        m = X.shape[0]
        
        # Calculate gradient for output layer
        dz2 = output - y
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Calculate gradient for hidden layer
        dz1 = np.dot(dz2, self.W2.T) * self.sigmoid_derivative(self.a1)
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        # Update weights and biases
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
    
    def compute_loss(self, y_true, y_pred):
        loss = -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))
        return loss
    
    def train(self, X, y, epochs, learning_rate):
        losses = []
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)
            loss = self.compute_loss(y, output)
            losses.append(loss)
            
            if (epoch + 1) % 200 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")
        
        return losses
    
    def predict(self, X):
        output = self.forward(X)
        return (output > 0.5).astype(int)

# ====== 4. Train Model ======
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1

mlp = MLP(input_size, hidden_size, output_size)
print("Training MLP with improved settings...\n")
losses = mlp.train(X_train, y_train, epochs=1000, learning_rate=0.1)

# ====== 5. Evaluation ======
y_pred_train = mlp.predict(X_train)
y_pred_test = mlp.predict(X_test)

train_accuracy = np.mean(y_pred_train == y_train)
test_accuracy = np.mean(y_pred_test == y_test)

print(f"\n✓ Train Accuracy: {train_accuracy:.4f}")
print(f"✓ Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# ====== Question 3: Compare Activation Functions ======

class MLP_MultiActivation:
    def __init__(self, input_size, hidden_size, output_size, activation='sigmoid'):
        self.activation_type = activation
        
        # Xavier initialization
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0/input_size)
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0/hidden_size)
        self.b2 = np.zeros((1, output_size))
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def sigmoid_derivative(self, z):
        return z * (1 - z)
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def relu_derivative(self, z):
        return (z > 0).astype(float)
    
    def tanh(self, z):
        return np.tanh(z)
    
    def tanh_derivative(self, z):
        return 1 - z**2
    
    def activate(self, z):
        if self.activation_type == 'sigmoid':
            return self.sigmoid(z)
        elif self.activation_type == 'relu':
            return self.relu(z)
        elif self.activation_type == 'tanh':
            return self.tanh(z)
    
    def activate_derivative(self, z):
        if self.activation_type == 'sigmoid':
            return self.sigmoid_derivative(z)
        elif self.activation_type == 'relu':
            return self.relu_derivative(z)
        elif self.activation_type == 'tanh':
            return self.tanh_derivative(z)
    
    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.activate(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)  # Output layer always sigmoid for binary classification
        return self.a2
    
    def backward(self, X, y, output, learning_rate):
        m = X.shape[0]
        
        # Output layer gradient
        dz2 = output - y
        dW2 = np.dot(self.a1.T, dz2) / m
        db2 = np.sum(dz2, axis=0, keepdims=True) / m
        
        # Hidden layer gradient
        dz1 = np.dot(dz2, self.W2.T) * self.activate_derivative(self.a1)
        dW1 = np.dot(X.T, dz1) / m
        db1 = np.sum(dz1, axis=0, keepdims=True) / m
        
        # Update weights
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
    
    def compute_loss(self, y_true, y_pred):
        loss = -np.mean(y_true * np.log(y_pred + 1e-8) + (1 - y_true) * np.log(1 - y_pred + 1e-8))
        return loss
    
    def train(self, X, y, epochs, learning_rate, verbose=False):
        losses = []
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)
            loss = self.compute_loss(y, output)
            losses.append(loss)
            
            if verbose and (epoch + 1) % 200 == 0:
                print(f"  Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")
        
        return losses
    
    def predict(self, X):
        output = self.forward(X)
        return (output > 0.5).astype(int)

# Train and compare different activation functions
activations = ['sigmoid', 'relu', 'tanh']
results = {}

print("=" * 60)
print("Comparing Activation Functions: Sigmoid vs ReLU vs Tanh")
print("=" * 60)

for activation in activations:
    print(f"\n--- Training with {activation.upper()} activation ---")
    
    model = MLP_MultiActivation(input_size, hidden_size, output_size, activation=activation)
    losses = model.train(X_train, y_train, epochs=1000, learning_rate=0.1, verbose=True)
    
    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Accuracy
    train_acc = np.mean(y_pred_train == y_train)
    test_acc = np.mean(y_pred_test == y_test)
    
    results[activation] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'final_loss': losses[-1],
        'losses': losses
    }
    
    print(f"✓ Train Accuracy: {train_acc:.4f}")
    print(f"✓ Test Accuracy: {test_acc:.4f}")

# Summary comparison
print("\n" + "=" * 60)
print("COMPARISON SUMMARY")
print("=" * 60)
print(f"{'Activation':<12} {'Train Acc':<12} {'Test Acc':<12} {'Final Loss':<12}")
print("-" * 60)
for activation in activations:
    r = results[activation]
    print(f"{activation.upper():<12} {r['train_accuracy']:<12.4f} {r['test_accuracy']:<12.4f} {r['final_loss']:<12.4f}")

# Find best activation
best_activation = max(results.items(), key=lambda x: x[1]['test_accuracy'])
print(f"\n🏆 Best Activation: {best_activation[0].upper()} with Test Accuracy = {best_activation[1]['test_accuracy']:.4f}")

In [ ]:
# ====== Question 4: Experiment with Learning Rates ======
import matplotlib.pyplot as plt

# Test different learning rates
learning_rates = [0.001, 0.01, 0.1]
lr_results = {}

print("=" * 60)
print("Experimenting with Different Learning Rates")
print("=" * 60)

for lr in learning_rates:
    print(f"\n--- Training with Learning Rate = {lr} ---")
    
    model = MLP(input_size, hidden_size, output_size)
    losses = model.train(X_train, y_train, epochs=1000, learning_rate=lr)
    
    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Accuracy
    train_acc = np.mean(y_pred_train == y_train)
    test_acc = np.mean(y_pred_test == y_test)
    
    lr_results[lr] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'losses': losses
    }
    
    print(f"✓ Train Accuracy: {train_acc:.4f}")
    print(f"✓ Test Accuracy: {test_acc:.4f}")
    print(f"✓ Final Loss: {losses[-1]:.4f}")

# Visualize loss curves
print("\n--- Visualizing Convergence Speed ---")
plt.figure(figsize=(12, 5))

# Plot 1: Loss curves
plt.subplot(1, 2, 1)
for lr in learning_rates:
    plt.plot(lr_results[lr]['losses'], label=f'LR = {lr}')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves for Different Learning Rates')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Accuracy comparison
plt.subplot(1, 2, 2)
lrs = [str(lr) for lr in learning_rates]
train_accs = [lr_results[lr]['train_accuracy'] for lr in learning_rates]
test_accs = [lr_results[lr]['test_accuracy'] for lr in learning_rates]

x = np.arange(len(lrs))
width = 0.35

plt.bar(x - width/2, train_accs, width, label='Train Accuracy', alpha=0.8)
plt.bar(x + width/2, test_accs, width, label='Test Accuracy', alpha=0.8)
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison')
plt.xticks(x, lrs)
plt.legend()
plt.ylim([0, 1])
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "=" * 60)
print("LEARNING RATE COMPARISON SUMMARY")
print("=" * 60)
print(f"{'LR':<10} {'Train Acc':<12} {'Test Acc':<12} {'Final Loss':<12}")
print("-" * 60)
for lr in learning_rates:
    r = lr_results[lr]
    print(f"{lr:<10} {r['train_accuracy']:<12.4f} {r['test_accuracy']:<12.4f} {r['losses'][-1]:<12.4f}")

# Find best learning rate
best_lr = max(lr_results.items(), key=lambda x: x[1]['test_accuracy'])
print(f"\n🏆 Best Learning Rate: {best_lr[0]} with Test Accuracy = {best_lr[1]['test_accuracy']:.4f}")

In [ ]:
# ====== Question 5: Analyze Hidden Layer Size Impact ======

# Test different hidden layer sizes
hidden_sizes = [16, 64, 128]
hidden_results = {}

print("=" * 60)
print("Analyzing Hidden Layer Size Impact")
print("=" * 60)

for h_size in hidden_sizes:
    print(f"\n--- Training with Hidden Size = {h_size} neurons ---")
    
    model = MLP(input_size, h_size, output_size)
    losses = model.train(X_train, y_train, epochs=1000, learning_rate=0.1)
    
    # Predictions
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Accuracy
    train_acc = np.mean(y_pred_train == y_train)
    test_acc = np.mean(y_pred_test == y_test)
    
    # Count parameters
    total_params = (input_size * h_size + h_size) + (h_size * output_size + output_size)
    
    hidden_results[h_size] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'losses': losses,
        'parameters': total_params
    }
    
    print(f"✓ Train Accuracy: {train_acc:.4f}")
    print(f"✓ Test Accuracy: {test_acc:.4f}")
    print(f"✓ Total Parameters: {total_params}")

# Visualize results
print("\n--- Visualizing Hidden Layer Impact ---")
plt.figure(figsize=(14, 5))

# Plot 1: Loss curves
plt.subplot(1, 3, 1)
for h_size in hidden_sizes:
    plt.plot(hidden_results[h_size]['losses'], label=f'{h_size} neurons')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curves for Different Hidden Sizes')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Accuracy comparison
plt.subplot(1, 3, 2)
sizes = [str(h) for h in hidden_sizes]
train_accs = [hidden_results[h]['train_accuracy'] for h in hidden_sizes]
test_accs = [hidden_results[h]['test_accuracy'] for h in hidden_sizes]

x = np.arange(len(sizes))
width = 0.35

plt.bar(x - width/2, train_accs, width, label='Train Accuracy', alpha=0.8)
plt.bar(x + width/2, test_accs, width, label='Test Accuracy', alpha=0.8)
plt.xlabel('Hidden Layer Size')
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison')
plt.xticks(x, sizes)
plt.legend()
plt.ylim([0, 1])
plt.grid(True, alpha=0.3, axis='y')

# Plot 3: Parameters vs Accuracy
plt.subplot(1, 3, 3)
params = [hidden_results[h]['parameters'] for h in hidden_sizes]
test_accs = [hidden_results[h]['test_accuracy'] for h in hidden_sizes]

plt.plot(params, test_accs, marker='o', linewidth=2, markersize=10)
for i, h in enumerate(hidden_sizes):
    plt.annotate(f'{h} neurons', (params[i], test_accs[i]), 
                textcoords="offset points", xytext=(0,10), ha='center')
plt.xlabel('Number of Parameters')
plt.ylabel('Test Accuracy')
plt.title('Model Complexity vs Performance')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "=" * 70)
print("HIDDEN LAYER SIZE COMPARISON SUMMARY")
print("=" * 70)
print(f"{'Hidden Size':<15} {'Parameters':<15} {'Train Acc':<12} {'Test Acc':<12}")
print("-" * 70)
for h_size in hidden_sizes:
    r = hidden_results[h_size]
    print(f"{h_size:<15} {r['parameters']:<15} {r['train_accuracy']:<12.4f} {r['test_accuracy']:<12.4f}")

# Find best hidden size
best_hidden = max(hidden_results.items(), key=lambda x: x[1]['test_accuracy'])
print(f"\n🏆 Best Hidden Size: {best_hidden[0]} neurons with Test Accuracy = {best_hidden[1]['test_accuracy']:.4f}")

# Analysis
print("\n" + "=" * 70)
print("ANALYSIS")
print("=" * 70)
overfitting_gap = []
for h_size in hidden_sizes:
    gap = hidden_results[h_size]['train_accuracy'] - hidden_results[h_size]['test_accuracy']
    overfitting_gap.append(gap)
    status = "Possible Overfitting" if gap > 0.05 else "Good Generalization"
    print(f"Hidden Size {h_size}: Train-Test Gap = {gap:.4f} → {status}")